In [ ]:
../julia-1.10.0/bin/julia --project=. --thread 32

In [ ]:

using HopfieldDCA, Revise
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H = 32; alg = HopPlmVar_gen(H, filepath);
n_ep = 3; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ, fig_save = true, );
println("test_done")


In [ ]:
using HopfieldDCA, Revise
folders = ["../DataAttentionDCA/data/PF00014/",
"../DataAttentionDCA/data/PF00035/",
"../DataAttentionDCA/data/PF00072/",
"../DataAttentionDCA/data/PF00076/",
"../DataAttentionDCA/data/PF00169/",
"../DataAttentionDCA/data/PF00595/",
"../DataAttentionDCA/data/PF00677/",
"../DataAttentionDCA/data/PF00684/",
"../DataAttentionDCA/data/PF00763/",
"../DataAttentionDCA/data/PF13354/"]; seq_paths = ["../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz",
       "../DataAttentionDCA/data/PF00035/PF00035_full.fasta",
       "../DataAttentionDCA/data/PF00072/PF00072_mgap6.fasta.gz",
       "../DataAttentionDCA/data/PF00076/PF00076_mgap6.fasta.gz",
       "../DataAttentionDCA/data/PF00169/PF00169_full.fasta",
       "../DataAttentionDCA/data/PF00595/PF00595_mgap6.fasta.gz",
       "../DataAttentionDCA/data/PF00677/PF00677_full.fasta",
       "../DataAttentionDCA/data/PF00684/PF00684_full.fasta",
       "../DataAttentionDCA/data/PF00763/PF00763_full.fasta",
       "../DataAttentionDCA/data/PF13354/PF13354_w0_ref_seqs.fasta.gz"]; structs = ["../DataAttentionDCA/data/PF00014/PF00014_struct.dat",
       "../DataAttentionDCA/data/PF00035/Atomic_distances_PF00035.dat",
       "../DataAttentionDCA/data/PF00072/PF00072_struct.dat",
       "../DataAttentionDCA/data/PF00076/PF00076_struct.dat",
       "../DataAttentionDCA/data/PF00169/Atomic_distances_PF00169.dat",
       "../DataAttentionDCA/data/PF00595/PF00595_struct.dat",
       "../DataAttentionDCA/data/PF00677/Atomic_distances_PF00677.dat",
       "../DataAttentionDCA/data/PF00684/Atomic_distances_PF00684.dat",
       "../DataAttentionDCA/data/PF00763/Atomic_distances_PF00763.dat",
       "../DataAttentionDCA/data/PF13354/PF13354_struct.dat"]; 
idx = 1; filepath = seq_paths[idx]; H = 32; alg = HopPlmVar_gen(H, filepath); η = 0.5; λ = 0.001;
n_ep = 2; @time m,n = trainer_J(alg, n_ep, η = η, λ = λ, 
    structfile = structs[idx],
    folder = folders[idx], 
    savefile = folders[idx], 
    fig_save = true, 
    par_save = true);


In [ ]:
#code for testing analytical gradient
using HopfieldDCA; using Tullio; import Flux: gradient;
filepath = "../DataAttentionDCA/data/PF00014/PF00014_mgap6.fasta.gz"; H =32; alg = HopPlmVar_gen(H, filepath;T=Float64); _w = alg.W ./ sum(alg.W); tmp = Stg(alg);
@time gt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@time gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
sum(gt.K[:].-gk[:])
sum(gt.V[:].-gv[:])
import BenchmarkTools: @btime
@btime gkt,gvt = get_anal_grad(alg.K, alg.V, alg.Z, _w, tmp);
@btime gk,gv = gradient((p1,p2)->get_loss_J(p1,p2, alg.Z, _w), alg.K, alg.V);
n_ep = 30; η = 0.5; λ = 0.001;
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
@time m,n = trainer_J(alg, n_ep, η = η, λ = λ);
@time m,n = trainer(alg, n_ep, η = η, λ = λ);
println("the end")


In [ ]:
structfile = "../DataAttentionDCA/data/PF00014/PF00014_struct.dat"
s = HopfieldDCA.score(m.K, m.V); PPV_h = HopfieldDCA.compute_PPV(s, structfile);

close("all"); plot(ppv_plm[1], label="Plm"); plot(PPV_h, label="HopPlm");legend();savefig("ciao.png");

